In [1]:
#import shit

import sys
import cv2

import matplotlib.pyplot as plt
import tensorflow as tf
import keras  
import os
import glob 
from skimage import io 
import random 
from skimage.color import rgb2hsv, rgb2gray, rgb2yuv
from skimage import color, exposure, transform
from skimage.exposure import equalize_hist


c:\Users\shady\anaconda3\envs\COMP9517\lib\site-packages\requests\__init__.py:114: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (2.3.0)/charset_normalizer (None) doesn't match a supported version!
  RequestsDependencyWarning,


In [33]:
#import keras stuff
from keras import applications
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Convolution2D,Activation,Flatten,Dense,Dropout,MaxPool2D,BatchNormalization
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19

In [34]:
sys.path.append('../elpv-dataset/utils')
from elpv_reader import load_dataset
images, probs, types = load_dataset()



In [35]:
import numpy as np
import pandas as pd

#print(images.shape,probs.shape,types.shape)
data = images

# Define degradation categories based on labels
def categorize_labels(labels):
    categories = []
    for label in labels:
        if label == 0 :
            categories.append(0)
        elif label <= 0.33:
            categories.append(1)
        elif label <= 0.66:
            categories.append(2)
        else:
            categories.append(3)
    return categories

label = categorize_labels(probs)

train_images_reshaped = data.reshape((-1, 300, 300))
train_labels_reshaped = to_categorical(label, num_classes=4)

print('\n',train_images_reshaped.shape, train_labels_reshaped.shape)


 (2624, 300, 300) (2624, 4)


In [13]:
# Convolution Neural Networks (CNN)
model = Sequential()

model.add(Conv2D(8, kernel_size=(5,5), padding='same', input_shape = (300,300,1)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(3, 3)))

model.add(Conv2D(16, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(4, activation='softmax'))
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 300, 300, 8)       208       
                                                                 
 activation_12 (Activation)  (None, 300, 300, 8)       0         
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 100, 100, 8)      0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 100, 100, 16)      1168      
                                                                 
 batch_normalization_9 (Batc  (None, 100, 100, 16)     64        
 hNormalization)                                                 
                                                                 
 activation_13 (Activation)  (None, 100, 100, 16)     

In [30]:
model.fit(train_images_reshaped, train_labels_reshaped, epochs=20)

Epoch 1/20
82/82 [==============================] - 2s 22ms/step - loss: 0.8285 - accuracy: 0.6502
Epoch 2/20
82/82 [==============================] - 2s 21ms/step - loss: 0.8275 - accuracy: 0.6456
Epoch 3/20
82/82 [==============================] - 2s 20ms/step - loss: 0.8292 - accuracy: 0.6460
Epoch 4/20
82/82 [==============================] - 2s 20ms/step - loss: 0.8254 - accuracy: 0.6502
Epoch 5/20
82/82 [==============================] - 2s 20ms/step - loss: 0.8215 - accuracy: 0.6547
Epoch 6/20
82/82 [==============================] - 2s 20ms/step - loss: 0.8229 - accuracy: 0.6490
Epoch 7/20
82/82 [==============================] - 2s 20ms/step - loss: 0.8166 - accuracy: 0.6578
Epoch 8/20
82/82 [==============================] - 2s 20ms/step - loss: 0.8184 - accuracy: 0.6524
Epoch 9/20
82/82 [==============================] - 2s 20ms/step - loss: 0.8173 - accuracy: 0.6517
Epoch 10/20
82/82 [==============================] - 2s 20ms/step - loss: 0.8133 - accuracy: 0.6536
Epoch 11/

In [31]:
model.save_weights('cnn_model1.h5')

In [32]:
from sklearn.model_selection import train_test_split
trainX, testX, trainy, testy = train_test_split(train_images_reshaped, train_labels_reshaped, test_size=0.2)
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '../elpv-dataset/images/',  
        target_size=(300, 300),  
        batch_size=batch_size,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=20)
model.save_weights('cnn_data_augmentation.h5')  # always save your weights after training or during training

Found 0 images belonging to 0 classes.


c:\Users\shady\anaconda3\envs\COMP9517\lib\site-packages\ipykernel_launcher.py:22: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [ ]:
batch_size = 1
test_generator = test_datagen.flow_from_directory(
    './TestData/',
    color_mode = "rgb",
    target_size=(100, 100),
    batch_size=1, 
    shuffle=True)
y_pred = model.predict_generator(test_generator, 1//batch_size, workers=4)

In [ ]:
x_test, y_test = next(test_generator)
p = model.predict(x_test)
print(p)
p = p.argmax()
plt.imshow(x_test[0])
plt.title(label_dict[p])

In [ ]:
model3 = Sequential()
model3.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(100,100,3)))
model3.add(Conv2D(8, (3, 3), activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.25))
model3.add(Flatten())
model3.add(Dense(32, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(4, activation='softmax'))

model3.summary()
model3.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model3.fit(train_images_reshaped, train_labels_reshaped, epochs=10)

#test_loss, test_acc = model3.evaluate(test_images_reshaped, test_labels_reshaped)

#print('Test accuracy:', test_acc)

In [ ]:
x_test, y_test = next(test_generator)
p = model3.predict(x_test)
print(p)
p = p.argmax()
plt.imshow(x_test[0])
plt.title(label_dict[p])

In [ ]:
model = applications.VGG16(include_top=False, weights='imagenet')
model.summary()

In [ ]:
 # dimensions of our images.  
img_width, img_height = 100, 100  
   
top_model_weights_path = 'bottleneck_fc_model.h5'  
train_data_dir = './train_data/'  
validation_data_dir = './TestData/'  
   
 # number of epochs to train top model  
epochs = 50  
 # batch size used by flow_from_directory and predict_generator  
batch_size = 16  

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)  
   
generator = datagen.flow_from_directory(  
     train_data_dir,  
     target_size=(img_width, img_height),  
     batch_size=batch_size,  
     class_mode=None,  
     shuffle=False)  

nb_train_samples = len(generator.filenames)  
num_classes = len(generator.class_indices)  

predict_size_train = int(math.ceil(nb_train_samples / batch_size))  
bottleneck_features_train = model.predict_generator(  
     generator, predict_size_train)  

In [ ]:
np.save('bottleneck_features_train.npy', bottleneck_features_train)
print(bottleneck_features_train.shape)

In [ ]:
generator.filenames[:5]

In [ ]:
generator.class_indices

In [ ]:
generator = datagen.flow_from_directory(  
    './TestData/',  
    target_size=(100, 100),  
    batch_size=batch_size,  
    class_mode=None,  
    shuffle=False)  
   
nb_validation_samples = len(generator.filenames)  
   
predict_size_validation = int(math.ceil(nb_validation_samples /batch_size))  
   
bottleneck_features_validation = model.predict_generator(  
    generator, predict_size_validation)  
   
np.save('bottleneck_features_validation.npy', bottleneck_features_validation)  

In [ ]:
print(bottleneck_features_validation.shape)

In [ ]:
datagen_top = ImageDataGenerator(rescale=1./255)  
generator_top = datagen_top.flow_from_directory(  
         train_data_dir,  
         target_size=(img_width, img_height),  
         batch_size=batch_size,  
         class_mode='categorical',  
         shuffle=False)  
   
nb_train_samples = len(generator_top.filenames)  
num_classes = len(generator_top.class_indices)  
   
train_data = np.load('bottleneck_features_train.npy')  
train_labels = generator_top.classes  
train_labels = to_categorical(train_labels, num_classes=num_classes)

In [ ]:
generator_top = datagen_top.flow_from_directory(  
         validation_data_dir,  
         target_size=(img_width, img_height),  
         batch_size=batch_size,  
         class_mode=None,  
         shuffle=False)  
   
nb_validation_samples = len(generator_top.filenames)  
   
validation_data = np.load('bottleneck_features_validation.npy')  
   
validation_labels = generator_top.classes  
validation_labels = to_categorical(validation_labels, num_classes=num_classes)    

In [ ]:
train_data.shape, train_labels.shape, validation_labels.shape

In [ ]:
model = Sequential()  
model.add(Flatten(input_shape=train_data.shape[1:]))  
model.add(Dense(256, activation='relu'))  
model.add(Dropout(0.5))  
model.add(Dense(num_classes, activation='softmax'))  
   
model.compile(optimizer='rmsprop',  
              loss='categorical_crossentropy', metrics=['accuracy'])  
   
history = model.fit(train_data, train_labels,  
          epochs=50,  
          batch_size=16,  
          validation_data=(validation_data, validation_labels))  
   
model.save_weights('top_model_weights.h5')  
   
(eval_loss, eval_accuracy) = model.evaluate(  
     validation_data, validation_labels, batch_size=batch_size, verbose=1)
print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))  
print("[INFO] Loss: {}".format(eval_loss))  

In [ ]:
plt.figure(1)  
   
# summarize history for accuracy  
   
plt.subplot(211)  
plt.plot(history.history['acc'])  
plt.plot(history.history['val_acc'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  

# summarize history for loss  

plt.subplot(212)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
plt.show() 

In [ ]:
def load_img(path):
    image_path = path

    orig = cv2.imread(image_path)  
    plt.imshow(orig)
    image = cv2.resize(orig, (100,100))  
    image = tf.keras.preprocessing.image.img_to_array(image)  

    image = image / 255  

    image = np.expand_dims(image, axis=0)
    return image

In [ ]:
def predict_img(image):
    # build the VGG16 network  
    model = applications.VGG16(include_top=False, weights='imagenet')  

    bottleneck_prediction = model.predict(image)  

    model = Sequential()  
    model.add(Flatten(input_shape=bottleneck_prediction.shape[1:]))  
    model.add(Dense(256, activation='relu'))  
    model.add(Dropout(0.5))  
    model.add(Dense(num_classes, activation='sigmoid'))  

    model.load_weights('./top_model_weights.h5')  

    class_predicted = model.predict_classes(bottleneck_prediction)  
    return label_dict[class_predicted[0]]

In [ ]:
predict_img(load_img('./TestData/image_chaises/image1.jpg' ))

In [ ]:
predict_img(load_img('./TestData/image_chaises/IMG_20180829_084101.jpg'))